In [1]:
from datasets import load_dataset, load_metric,Dataset,DatasetDict
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
from transformers import AutoTokenizer
import nltk
import numpy as np

#datasets = ["samsum","cnn","xsum"]
model_name = "facebook/bart-large"

In [2]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]
    
    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    # Extract a few results
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    
    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    
    return {k: round(v, 4) for k, v in result.items()}

In [3]:
raw_datasets = load_dataset("samsum")
metric = load_metric("rouge")
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)



Using the latest cached version of the module from /home/weichen/.cache/huggingface/modules/datasets_modules/datasets/samsum/3f7dba43be72ab10ca66a2e0f8547b3590e96c2bd9f2cbb1f6bb1ec1f1488ba6 (last modified on Wed Apr 28 13:59:29 2021) since it couldn't be found locally at samsum/samsum.py or remotely (ConnectionError).
Reusing dataset samsum (/home/weichen/.cache/huggingface/datasets/samsum/samsum/0.0.0/3f7dba43be72ab10ca66a2e0f8547b3590e96c2bd9f2cbb1f6bb1ec1f1488ba6)
Using the latest cached version of the module from /home/weichen/.cache/huggingface/modules/datasets_modules/metrics/rouge/2b73d5eb463209373e9d21a95decb226d4164bdca4c361b8dfad295ec82bc62e (last modified on Wed Apr 28 14:00:09 2021) since it couldn't be found locally at rouge/rouge.py or remotely (ConnectionError).


In [4]:
max_input_length = 512
max_target_length = 128

def preprocess_function(examples):
    inputs = [doc for doc in examples["dialogue"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["summary"], max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [5]:
tokenized_datasets = raw_datasets.map(preprocess_function, batched=True)

In [6]:
batch_size = 8
args = Seq2SeqTrainingArguments(
    "BART-LARGE-samsum",
    evaluation_strategy = "steps",
    eval_steps = 500,
    learning_rate=3e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=5,
    predict_with_generate=True,
    fp16=True,
    warmup_steps=200,
    load_best_model_at_end=True,
    metric_for_best_model="eval_rouge1",
    greater_is_better=True
)

In [7]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [8]:
trainer.train()

/home/weichen/anaconda3/envs/emotion_fan/lib/python3.9/site-packages/torch/optim/lr_scheduler.py:129: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "


Step,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len,Runtime,Samples Per Second
500,1.828700,1.504997,46.025300,23.523500,38.553800,42.304800,18.026900,55.408700,14.763000
1000,1.613700,1.471320,47.186800,24.570500,39.975800,43.813200,18.385100,55.597300,14.713000
1500,1.575400,1.420214,47.280800,25.574200,40.398900,43.660900,17.316600,55.455400,14.751000
2000,1.450400,1.412556,48.142600,25.769800,41.345100,44.688700,17.773800,55.323400,14.786000
2500,1.307000,1.411188,47.695900,25.543900,40.645600,44.034200,17.602700,55.450100,14.752000
3000,1.312300,1.400580,48.829100,26.624200,41.821600,44.910500,17.713900,55.284100,14.796000
3500,1.284800,1.405458,49.305700,26.945800,41.910100,46.008900,17.734700,55.332000,14.783000
4000,1.156000,1.408802,49.271600,26.830200,41.945300,45.573200,18.039100,55.352000,14.778000
4500,1.063800,1.421972,48.315800,26.333400,41.347900,44.759900,18.540300,55.331900,14.784000
5000,1.082400,1.384098,49.703100,27.081900,42.212400,45.675000,18.512200,55.800100,14.659000


TrainOutput(global_step=9210, training_loss=1.144713898202107, metrics={'train_runtime': 3843.2667, 'train_samples_per_second': 2.396, 'total_flos': 7.023151183117517e+16, 'epoch': 5.0, 'init_mem_cpu_alloc_delta': 1585229824, 'init_mem_gpu_alloc_delta': 1625367040, 'init_mem_cpu_peaked_delta': 1621676032, 'init_mem_gpu_peaked_delta': 0, 'train_mem_cpu_alloc_delta': 3025321984, 'train_mem_gpu_alloc_delta': 6509939712, 'train_mem_cpu_peaked_delta': 411684864, 'train_mem_gpu_peaked_delta': 8868905984})

In [9]:
out = trainer.predict(tokenized_datasets["test"],num_beams=5)

In [10]:
print(out)

PredictionOutput(predictions=array([[    2,     0,     0, ...,  1394,  6045,     2],
       [    2, 24375,     8, ...,     1,     1,     1],
       [    2,     0,     0, ...,  3045,     4,     2],
       ...,
       [    2,     0, 13012, ...,    30,  3213,     2],
       [    2,     0,     0, ...,    11,  8353,     2],
       [    2, 41415, 11210, ...,   416,  3996,     2]]), label_ids=array([[    0,   725, 25984, ...,  -100,  -100,  -100],
       [    0, 24375,     8, ...,  -100,  -100,  -100],
       [    0,   574, 11867, ...,  -100,  -100,  -100],
       ...,
       [    0, 13012,   102, ...,  -100,  -100,  -100],
       [    0,   970,    21, ...,  -100,  -100,  -100],
       [    0, 41415, 11210, ...,  -100,  -100,  -100]]), metrics={'eval_loss': 1.4359217882156372, 'eval_rouge1': 48.3409, 'eval_rouge2': 25.6455, 'eval_rougeL': 41.0306, 'eval_rougeLsum': 44.3759, 'eval_gen_len': 17.8181, 'eval_runtime': 59.4663, 'eval_samples_per_second': 13.773, 'test_mem_cpu_alloc_delta': -117116